In [ ]:
import sys
sys.path.append("../")

import sympy as sp
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import backend_source_dir
from affine_body_core import compute_J_point, compute_J_vec

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")
Gen.DisableLatexComment()

ref: [Affine Body Prismatic Joint](https://spirimirror.github.io/libuipc-doc/specification/constitutions/affine_body_prismatic_joint/)

![Affine Body Prismatic Joint](../../docs/specification/constitutions/media/affine_body_prismatic_joint_fig1.drawio.svg)

$
\begin{aligned}
C_0 &= (\mathbf{c}_j - \mathbf{c}_i) \times \hat{\mathbf{t}}_i &= \mathbf{0} \\
C_1 &= (\mathbf{c}_i - \mathbf{c}_j) \times \hat{\mathbf{t}}_j &= \mathbf{0} \\
C_2 &= \hat{\mathbf{n}}_i - \hat{\mathbf{n}}_j &= 0 \\
C_3 &= \hat{\mathbf{b}}_i - \hat{\mathbf{b}}_j &= 0
\end{aligned}
$$

In [ ]:
kappa  = Eigen.Scalar("kappa")

# Basis vectors
ci_bar = Eigen.Vector("ci_bar",3)
ti_bar = Eigen.Vector("ti_bar",3)
ni_bar = Eigen.Vector("ni_bar",3)
bi_bar = Eigen.Vector("bi_bar",3)

cj_bar = Eigen.Vector("cj_bar",3)
tj_bar = Eigen.Vector("tj_bar",3)
nj_bar = Eigen.Vector("nj_bar",3)
bj_bar = Eigen.Vector("bj_bar",3)

# Affine Body DOF vectors
qi = Eigen.Vector("qi",12)
qj = Eigen.Vector("qj",12)

$$
C_0 = (\mathbf{c}_j - \mathbf{c}_i) \times \hat{\mathbf{t}}_i = \mathbf{0} \\
C_1 = (\mathbf{c}_i - \mathbf{c}_j) \times \hat{\mathbf{t}}_j = \mathbf{0}
$$

$$
\mathbf{F}_{01} = \begin{bmatrix}
\mathbf{c}_j - \mathbf{c}_i \\
\hat{\mathbf{t}}_i \\
\hat{\mathbf{t}}_j \\
\end{bmatrix}_{9 \times 1}
$$

Frame Affine Body Mapping:

$$
\mathbf{J}_0 = \begin{bmatrix}
-\mathbf{J}(\bar{\mathbf{c}}_i) & \mathbf{J}(\bar{\mathbf{c}}_j) \\
\mathring{\mathbf{J}}(\bar{\mathbf{t}}_i) & \mathbf{0} \\
\mathbf{0} & \mathring{\mathbf{J}}(\bar{\mathbf{t}}_j) \\
\end{bmatrix}_{9 \times 24}
$$

$$
\mathbf{F}_{01} = \mathbf{J}_{01} \cdot
\begin{bmatrix}
\mathbf{q}_i \\
\mathbf{q}_j \\
\end{bmatrix}
$$

In [ ]:
# Mapping
J01 = sp.Matrix.zeros(9,24)
J01[0:3, 0:12] = -compute_J_point(ci_bar)
J01[0:3, 12:24] = compute_J_point(cj_bar)
J01[3:6, 0:12] = compute_J_vec(ti_bar)
J01[6:9, 12:24] = compute_J_vec(tj_bar)

content = ""

# from ABD q to F
F0_q = J01 @ sp.Matrix.vstack(qi, qj)
Cl_F01 = Gen.Closure(ci_bar, ti_bar, qi, # Affine Body i
                     cj_bar, tj_bar, qj) # Affine Body j
# from F Gradient to ABD Gradient
G9 = Eigen.Vector("G9",9)
J01T_G01 = J01.T @ G9
Cl_G01 = Gen.Closure(G9, 
                     ci_bar, ti_bar, # Affine Body i
                     cj_bar, tj_bar) # Affine Body j
# from F Hessian to ABD Hessian
H9x9 = Eigen.Matrix("H9x9",9,9)
J01T_H01_J01 = J01.T @ H9x9 @ J01
Cl_H01 = Gen.Closure(H9x9, 
                     ci_bar, ti_bar, # Affine Body i
                     cj_bar, tj_bar) # Affine Body j

content += f""" 
// Prismatic Joint: C0 C1
// Mapping between ABD qi qj to F01

{Cl_F01("F01", F0_q)}
{Cl_G01("J01T_G01", J01T_G01)}
{Cl_H01("J01T_H01_J01", J01T_H01_J01)}

"""

F01 = Eigen.Vector("F01", 9)
dij = F01[0:3, 0]
ti = F01[3:6, 0]
tj = F01[6:9, 0]

C0 = dij.cross(ti)
C1 = (-dij).cross(tj)
E01 = kappa * (C0.dot(C0) + C1.dot(C1))/2
# E01 = 0
Cl_E01 = Gen.Closure(kappa, F01)


dE01dF01 = VecDiff(E01, F01)
ddE01ddF01 = VecDiff(dE01dF01, F01)

content += f""" // Prismatic Joint Energy and Derivatives

{Cl_E01("E01", E01)}
{Cl_E01("dE01dF01", dE01dF01)}
{Cl_E01("ddE01ddF01", ddE01ddF01)}

// -------------------------------------------------------------------------
"""

$$
C_2 = \hat{\mathbf{n}}_i - \hat{\mathbf{n}}_j = 0
$$

$$
C_3 = \hat{\mathbf{b}}_i - \hat{\mathbf{b}}_j = 0
$$

No need to do make semi-definite Hessian since they are already semi-definite, thus we can compute energy derivatives directly without chain rule from F to C.

Frame Affine Body Mapping:

$$
\mathbf{J}_{23} = \begin{bmatrix}
\mathring{\mathbf{J}}(\bar{\mathbf{n}}_i) & -\mathring{\mathbf{J}}(\bar{\mathbf{n}}_j) \\
\mathring{\mathbf{J}}(\bar{\mathbf{b}}_i) & -\mathring{\mathbf{J}}(\bar{\mathbf{b}}_j) \\
\end{bmatrix}_{6 \times 24}
$$

$$
\mathbf{F}_{23} = \mathbf{J}_{23} \cdot
\begin{bmatrix}
\mathbf{q}_i \\
\mathbf{q}_j \\
\end{bmatrix}
$$

In [ ]:
J23 = sp.Matrix.zeros(6,24)
J23[0:3, 0:12] = compute_J_vec(ni_bar)
J23[0:3, 12:24] = - compute_J_vec(nj_bar)
J23[3:6, 0:12] = compute_J_vec(bi_bar)
J23[3:6, 12:24] = - compute_J_vec(bj_bar)

F23 = J23 @ sp.Matrix.vstack(qi, qj)

nij = F23[0:3, 0]
bij = F23[3:6, 0]

E23 = kappa * (nij.dot(nij) + bij.dot(bij))/2
Cl_E23 = Gen.Closure(kappa,
                     ni_bar, bi_bar, qi, # Affine Body i
                     nj_bar, bj_bar, qj) # Affine Body j

dE23dQ = VecDiff(E23, sp.Matrix.vstack(qi, qj))
ddE23ddQ = VecDiff(dE23dQ, sp.Matrix.vstack(qi, qj))

content += f""" // Prismatic Joint: C2 C3 Energy and Derivatives
{Cl_E23("E23", E23)}
{Cl_E23("dE23dQ", dE23dQ)}
{Cl_E23("ddE23ddQ", ddE23ddQ)}
// -------------------------------------------------------------------------
"""

In [ ]:
path = backend_source_dir("cuda") / "affine_body/constitutions/sym" / "affine_body_prismatic_joint.inl"
f = open(path, "w")
f.write(content)
f.close()
print(f"Written to {path}")